# RIDGE REGRESSION    
--- 
### Reason for using Ridge Regression: 
+ As an initial step of modeling, we are using three algorithms (Ridge Regression, LASSO, and PCR - Principal Component Regression) to look at the predictors that has a good impact in predicting the crop yield. 
+ We are using Ridge Regression mainly because of two reasons:      
    + We have 161 predictors and 42007 records in the dataset. But when we are modeling on Eco-district level this ratio is very small. There is not much difference between the number of predictors and the observations.     
    + As we have a lot of predictors there's a high chance of collinearity. It eliminates multicollinearity.   
+ It introduces a little bias so that the variance can be substantially reduced, which leads to a lower overall MSE.    
+ It is a Regularization algorithm. 
--- 
### Overview of what has been done for modeling in this file: 
+ In ridge regression the first step is to **standardize** the variables. All ridge regression calculations are based on standardized variables.    
+ We also did **cross-validation** for ridge regression, to test the model's ability to predict new data that was not used in estimating it, in order to flag problems like overfitting or selection bias and to give an insight on how the model will generalize to an independent dataset (new dataset). 
+ Then we have calculated the Train MSE (Mean Squared Error), Test MSE, R Squared for training set, R squared for test set, Mean Absolute Error (MAE), and Accuracy for each model for each eco district. 

--- 

**General information**: Initially when we did data exploration, we found that there are 144 eco districts. After data wrangling, joining the four datasets into one final dataset we are left with 137 eco districts. So, the further analysis is carried out for 137 eco districts.     

**NOTE: We have added proper comments for the first model, that  is the model for the first eco district. All of the other model code is similar to it.** 

---
---

## Code: 

In [1]:
# importing all of the required libraries. 
# various functions are imported from different libraries and packages. 

import pandas as pd    # pandas
import numpy as np     # numpy
from sklearn import model_selection   # from sklearn package imported model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import scale
from math import sqrt

In [2]:
dataframe = pd.read_csv("aafc_data.csv")  # importing the dataset
# summarize shape
print(dataframe.shape)
print("\n")
# summarize first few lines
dataframe.head()

(42007, 166)




,Unnamed: 0,TWP_ID,ECODISTRICT_ID,YEAR,YieldKgAcre,SumPcpn18_20,SumPcpn19_21,SumPcpn20_22,SumPcpn21_23,SumPcpn22_24,...,SoilMoisture29_31,SoilMoisture30_32,SoilMoisture31_33,SoilMoisture32_34,SoilMoisture33_35,SoilMoisture34_36,SoilMoisture35_37,SoilMoisture36_38,SoilMoisture37_39,SoilMoisture38_40
0,0,00101E1,852.0,2010,867.766846,53.6,111.1,109.7,117.9,46.4,...,16.960125,18.766207,17.186998,15.461519,19.738222,22.958089,27.206203,26.480087,28.678156,26.308484
1,1,00101W1,852.0,2010,673.685028,57.2,114.7,110.5,114.0,46.2,...,16.328520,17.926029,16.787544,14.779726,20.245149,23.608204,28.560990,27.324254,29.079177,26.927224
2,2,00101W2,796.0,2010,824.303864,39.0,96.4,109.8,101.2,111.4,...,13.117879,12.869142,12.831834,14.126196,16.385776,18.650751,20.287069,20.514132,19.564788,16.681692
3,3,00102E1,853.0,2010,1006.708496,37.5,158.2,157.8,161.4,46.9,...,17.060778,18.699156,17.345822,15.998957,20.091525,22.761273,26.337430,25.559602,27.611729,25.575794
4,4,00102W1,852.0,2010,869.040283,57.2,114.7,110.5,114.0,46.2,...,16.050993,17.556860,16.612026,14.480150,20.467884,23.893858,29.156274,27.695178,29.255386,27.199097


In [3]:
# dropping those columns that are not useful
dataframe.drop(['Unnamed: 0', 'TWP_ID', 'YEAR'], axis=1, inplace=True)

In [4]:
# looking at the dataset after dropping the columns 
dataframe.head()

,ECODISTRICT_ID,YieldKgAcre,SumPcpn18_20,SumPcpn19_21,SumPcpn20_22,SumPcpn21_23,SumPcpn22_24,SumPcpn23_25,SumPcpn24_26,SumPcpn25_27,...,SoilMoisture29_31,SoilMoisture30_32,SoilMoisture31_33,SoilMoisture32_34,SoilMoisture33_35,SoilMoisture34_36,SoilMoisture35_37,SoilMoisture36_38,SoilMoisture37_39,SoilMoisture38_40
0,852.0,867.766846,53.6,111.1,109.7,117.9,46.4,69.3,60.0,44.6,...,16.960125,18.766207,17.186998,15.461519,19.738222,22.958089,27.206203,26.480087,28.678156,26.308484
1,852.0,673.685028,57.2,114.7,110.5,114.0,46.2,68.1,55.9,34.9,...,16.328520,17.926029,16.787544,14.779726,20.245149,23.608204,28.560990,27.324254,29.079177,26.927224
2,796.0,824.303864,39.0,96.4,109.8,101.2,111.4,153.0,163.6,98.8,...,13.117879,12.869142,12.831834,14.126196,16.385776,18.650751,20.287069,20.514132,19.564788,16.681692
3,853.0,1006.708496,37.5,158.2,157.8,161.4,46.9,79.5,67.5,40.4,...,17.060778,18.699156,17.345822,15.998957,20.091525,22.761273,26.337430,25.559602,27.611729,25.575794
4,852.0,869.040283,57.2,114.7,110.5,114.0,46.2,68.1,55.9,34.9,...,16.050993,17.556860,16.612026,14.480150,20.467884,23.893858,29.156274,27.695178,29.255386,27.199097


---
### For the top 10 (which have large number of records) Eco districts, we have modeled them separately.   


| Ecodistrict | Count of Unique Township IDs |Count of Records |     
| :----: | :-----: | :-----: |
|748| 101 | 1199|    
|826|107|1177|    
|752|101|1111|
|745|100|1100|
|808|97|1067|
|792|92|1012|
|849|91|1001|
|729|86|946|
|753|86|946|
|709|83|913|

---
**NOTE: We have added proper comments for the first model, that  is the model for the first eco district. All of the other model code is similar to it.**   
### # For Eco District ID: 748         


In [5]:
df1 = dataframe[dataframe['ECODISTRICT_ID']==748]   # created a dataframe for one of the eco district
df1.drop(['ECODISTRICT_ID'], axis=1, inplace=True)  # dropped the 'ECODISTRICT_ID' column before modeling

# split data into X and y
x = pd.DataFrame(df1.drop(labels=['YieldKgAcre'], axis=1)) # x contains the predictors (not the target value 'YieldKgAcre')
y = pd.DataFrame(df1['YieldKgAcre'])                       # y contains the dependent variable ('YieldKgAcre').

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [6]:
# splitting the x and y datasets to train and test with the ratio 70:30 ratio.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)

# looking at the splitted test and train data shapes
print(X_train.shape); print(X_test.shape)


(839, 161)
(360, 161)


In [7]:
rr = Ridge(alpha=0.01)  # defining the model

# applying cross validation      
# Parameters: 
# 1. Alpha: Array of alpha values to try. Regularization strength; must be a positive float. 
#           Regularization improves the conditioning of the problem and reduces the variance of the estimates. 
#           Larger values specify stronger regularization.    
# 2. cv: Determines the cross-validation splitting strategy.
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)  


# fitting the model on training data
rr.fit(X_train, y_train)    

pred_train_rr= rr.predict(X_train)  # predicting the 'YieldKgAcre' values on the basis of trained model. Using the predict() function
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr))) # calculated the Training MSE using mean_squared_error() function.
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)  # calculated the R squared for the training set using r2_score() function.

pred_test_rr= rr.predict(X_test) # predicting the 'YieldKgAcre' values on the basis of trained model for the test dataset. Using the predict() function.
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) # calculated the Test MSE using mean_squared_error() function.
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)   # calculated the R squared for the test set using r2_score() function.






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)

# Calculate and display the accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  76.1919905362895
R squared training set:  84.72096804187296
Test MSE:  87.90676524170678
R squared test set:  80.78561045557035
Mean Absolute Error: YieldKgAcre    69.988648
dtype: float64 degrees.
Accuracy: YieldKgAcre    90.11
dtype: float64 %.


---
### # For Eco District ID: 826  

In [8]:
df2 = dataframe[dataframe['ECODISTRICT_ID']==826]
df2.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df2.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df2['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)

print(X_train.shape); print(X_test.shape)

(823, 161)
(354, 161)


In [10]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  101.09165827651603
R squared training set:  75.60317443634008
Test MSE:  105.57282197153029
R squared test set:  73.57893809551786
Mean Absolute Error: YieldKgAcre    82.970694
dtype: float64 degrees.
Accuracy: YieldKgAcre    87.34
dtype: float64 %.


---
### # For Eco District ID: 752

In [11]:
df3 = dataframe[dataframe['ECODISTRICT_ID']==752]
df3.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df3.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df3['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [12]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)

print(X_train.shape); print(X_test.shape)

(777, 161)
(334, 161)


In [13]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  78.66255663549335
R squared training set:  86.53688382863933
Test MSE:  92.89526127482065
R squared test set:  77.86303456647643
Mean Absolute Error: YieldKgAcre    71.969368
dtype: float64 degrees.
Accuracy: YieldKgAcre    90.07
dtype: float64 %.


---
### # For Eco District ID: 745  

In [14]:
df4 = dataframe[dataframe['ECODISTRICT_ID']==745]
df4.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df4.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df4['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)


print(X_train.shape); print(X_test.shape)

(770, 161)
(330, 161)


In [16]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  74.30969520931816
R squared training set:  89.2187411341816
Test MSE:  83.60354856900985
R squared test set:  87.42100132377485
Mean Absolute Error: YieldKgAcre    64.359002
dtype: float64 degrees.
Accuracy: YieldKgAcre    91.73
dtype: float64 %.


---
### # For Eco District ID: 808  

In [17]:
df5 = dataframe[dataframe['ECODISTRICT_ID']==808]
df5.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df5.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df5['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   
   
# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)     
        
        
print(X_train.shape); print(X_test.shape)

(746, 161)
(321, 161)


In [19]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  108.80776688895307
R squared training set:  71.47017415604348
Test MSE:  125.33205832535845
R squared test set:  60.32007296901117
Mean Absolute Error: YieldKgAcre    93.876136
dtype: float64 degrees.
Accuracy: YieldKgAcre    87.32
dtype: float64 %.


---
### # For Eco District ID: 792

In [20]:
df6 = dataframe[dataframe['ECODISTRICT_ID']==792]
df6.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df6.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df6['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [21]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)  

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)
  
    
print(X_train.shape); print(X_test.shape)

(708, 161)
(304, 161)


In [22]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  95.13732923966857
R squared training set:  78.53762506583996
Test MSE:  107.29538676736196
R squared test set:  72.22176207981319
Mean Absolute Error: YieldKgAcre    81.675159
dtype: float64 degrees.
Accuracy: YieldKgAcre    89.82
dtype: float64 %.


---
### # For Eco District ID: 849

In [23]:
df7 = dataframe[dataframe['ECODISTRICT_ID']==849]
df7.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df7.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df7['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [24]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   
  
# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)     
        
        
print(X_train.shape); print(X_test.shape)

(700, 161)
(301, 161)


In [25]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  78.54796667665148
R squared training set:  90.73648086893138
Test MSE:  95.63859918528432
R squared test set:  85.97108872906384
Mean Absolute Error: YieldKgAcre    71.924404
dtype: float64 degrees.
Accuracy: YieldKgAcre    89.99
dtype: float64 %.


---
### # For Eco District ID: 729

In [26]:
df8 = dataframe[dataframe['ECODISTRICT_ID']==729]
df8.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df8.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df8['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [27]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test) 
     
         
print(X_train.shape); print(X_test.shape)

(662, 161)
(284, 161)


In [28]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  71.19193192790989
R squared training set:  81.91779666344638
Test MSE:  98.91196333229948
R squared test set:  67.7959532281787
Mean Absolute Error: YieldKgAcre    75.362232
dtype: float64 degrees.
Accuracy: YieldKgAcre    91.5
dtype: float64 %.


---
### # For Eco District ID: 753  

In [29]:
df9 = dataframe[dataframe['ECODISTRICT_ID']==753]
df9.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df9.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df9['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [30]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   
  
# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)  

print(X_train.shape); print(X_test.shape)

(662, 161)
(284, 161)


In [31]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  58.01717143917308
R squared training set:  89.16091262673463
Test MSE:  70.3129400188808
R squared test set:  84.31207850205715
Mean Absolute Error: YieldKgAcre    56.129276
dtype: float64 degrees.
Accuracy: YieldKgAcre    93.31
dtype: float64 %.


---
### # For Eco District ID: 709  

In [32]:
df10 = dataframe[dataframe['ECODISTRICT_ID']==709]
df10.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
# split data into X and y
x = pd.DataFrame(df10.drop(labels=['YieldKgAcre'], axis=1))
y = pd.DataFrame(df10['YieldKgAcre'])

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [33]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   

# Run standardization on X variables
X_train = scale(X_train)
X_test = scale(X_test)
     
print(X_train.shape); print(X_test.shape)

(639, 161)
(274, 161)


In [34]:
rr = Ridge(alpha=0.01)
rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
rr.fit(X_train, y_train) 
pred_train_rr= rr.predict(X_train)
print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

pred_test_rr= rr.predict(X_test)
print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)






# Calculate the absolute errors
errors = abs(pred_test_rr - y_test)

 # Print out the mean absolute error (mae)
print('Mean Absolute Error:', np.mean(errors), 'degrees.')



# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Train MSE:  84.4591216123592
R squared training set:  87.89976256831535
Test MSE:  104.67585292319163
R squared test set:  82.28193554839056
Mean Absolute Error: YieldKgAcre    80.437732
dtype: float64 degrees.
Accuracy: YieldKgAcre    90.28
dtype: float64 %.


---
---
+ **Now, we will apply three for loops to get the MSEs, R-squared values, MAE, and the accuracy for the rest of the eco-districts.**      
+ **Finally, we will be able to see when the accuracy drops and decide the threshold to stop for the modeling.**  

In [35]:
# created a list of eco districts to be used in the 1st for loop
eco_district_ids_list1 = [782,825,760,767,794,749,816,830,756,765,770,773,707,724,795,736,763,780,832,723,783,696,733,822,754,687,758,766,
                          726,717,701,705,850,755,803,706,813,741,789,841,852,702,735,375,689,714,839,693,711,694,784,820,821,831,751,680,704,
                          757,817,838,715,747,776,807,690]

In [36]:
# created another list of eco districts to be used in the 2nd for loop 
eco_district_ids_list2 = [697,695,698,710,796,809,840,854,685,700,771,805,810,824,844,734,762,764,774,785,851,775,716,739,843,661,682,847,669,815,846,
       772,837,742,761,853,672,699,768,778,657,677,691,743,759,827,660,819,848,652,718,686,720]

In [37]:
# created another list of eco districts to be used in the 3rd for loop. 
# These eco districts have very less number of records.
# We have used cv=5 for these eco districts.
eco_district_ids_list3 = [811,855,379,647,659,662,668,833,834]

### # First for loop for modeling for the eco districts in the 1st list. 

In [38]:
for i in eco_district_ids_list1:   # for loop for which iterates through each eco district id in the 1st list.
    print("Summary for the Eco District ID:", i)   # printing the eco district ID
    
    # The modeling code is similar to what we have done before in this file.  
    
    df9 = dataframe[dataframe['ECODISTRICT_ID']==i]
    df9.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
    # split data into X and y
    x = pd.DataFrame(df9.drop(labels=['YieldKgAcre'], axis=1))
    y = pd.DataFrame(df9['YieldKgAcre'])
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   

    # Run standardization on X variables
    X_train = scale(X_train)
    X_test = scale(X_test)
     
    rr = Ridge(alpha=0.01)
    rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
    rr.fit(X_train, y_train) 
    pred_train_rr= rr.predict(X_train)
    print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
    print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

    pred_test_rr= rr.predict(X_test)
    print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
    print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)

    # Calculate the absolute errors
    errors = abs(pred_test_rr - y_test)

    # Print out the mean absolute error (mae)
    print('Mean Absolute Error:', np.mean(errors), 'degrees.')



    # Calculate mean absolute percentage error (MAPE)
    mape = 100 * (errors / y_test)
    # Calculate and display accuracy
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 2), '%.')
    print("\n")
    print("\n")

Summary for the Eco District ID: 782
Train MSE:  73.38728951043555
R squared training set:  74.87542579446689
Test MSE:  79.92267342047288
R squared test set:  75.49278534579338
Mean Absolute Error: YieldKgAcre    62.013096
dtype: float64 degrees.
Accuracy: YieldKgAcre    91.62
dtype: float64 %.




Summary for the Eco District ID: 825


C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Train MSE:  101.94780596272146
R squared training set:  71.06056475475584
Test MSE:  107.45455398170826
R squared test set:  70.17855644822903
Mean Absolute Error: YieldKgAcre    82.541958
dtype: float64 degrees.
Accuracy: YieldKgAcre    88.16
dtype: float64 %.




Summary for the Eco District ID: 760
Train MSE:  68.25776560815834
R squared training set:  83.37844977534044
Test MSE:  78.9628443209741
R squared test set:  78.19727071184695
Mean Absolute Error: YieldKgAcre    61.849039
dtype: float64 degrees.
Accuracy: YieldKgAcre    91.2
dtype: float64 %.




Summary for the Eco District ID: 767


C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Train MSE:  74.13721632869554
R squared training set:  81.20664943261424
Test MSE:  78.50980174701876
R squared test set:  79.65978225909683
Mean Absolute Error: YieldKgAcre    61.076218
dtype: float64 degrees.
Accuracy: YieldKgAcre    92.47
dtype: float64 %.




Summary for the Eco District ID: 794
Train MSE:  85.48554531488693
R squared training set:  76.62423463925943
Test MSE:  92.36910297978065
R squared test set:  69.7757362857084
Mean Absolute Error: YieldKgAcre    71.19386
dtype: float64 degrees.
Accuracy: YieldKgAcre    87.82
dtype: float64 %.




Summary for the Eco District ID: 749


...

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Train MSE:  92.41242692545985
R squared training set:  85.43591868273658
Test MSE:  116.79568200932405
R squared test set:  72.53168901986483
Mean Absolute Error: YieldKgAcre    99.582758
dtype: float64 degrees.
Accuracy: YieldKgAcre    86.43
dtype: float64 %.




Summary for the Eco District ID: 776
Train MSE:  76.18981479453473
R squared training set:  77.40801375226508
Test MSE:  126.35003795746113
R squared test set:  52.492266749021375
Mean Absolute Error: YieldKgAcre    103.834055
dtype: float64 degrees.
Accuracy: YieldKgAcre    86.81
dtype: float64 %.




Summary for the Eco District ID: 807
Train MSE:  92.95206049833237
R squared training set:  73.06743402020201
Test MSE:  104.80237933837796
R squared test set:  62.48074682440987
Mean Absolute Error:

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


 YieldKgAcre    85.617267
dtype: float64 degrees.
Accuracy: YieldKgAcre    88.88
dtype: float64 %.




Summary for the Eco District ID: 690
Train MSE:  72.40653350051025
R squared training set:  85.97173660603734
Test MSE:  115.84336944673001
R squared test set:  65.48769958919904
Mean Absolute Error: YieldKgAcre    83.986691
dtype: float64 degrees.
Accuracy: YieldKgAcre    90.01
dtype: float64 %.






C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### # Second for loop for modeling for the eco districts in the 2nd list. 

In [39]:
for i in eco_district_ids_list2:       # for loop for which iterates through each eco district id in the 2nd list.
    print("Summary for the Eco District ID:", i)  # printing the eco district ID
    
    
    # The modeling code is similar to what we have done before in this file. 
    df9 = dataframe[dataframe['ECODISTRICT_ID']==i]
    df9.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
    # split data into X and y
    x = pd.DataFrame(df9.drop(labels=['YieldKgAcre'], axis=1))
    y = pd.DataFrame(df9['YieldKgAcre'])
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   

    # Run standardization on X variables
    X_train = scale(X_train)
    X_test = scale(X_test)
     
    rr = Ridge(alpha=0.01)
    rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=10)
    rr.fit(X_train, y_train) 
    pred_train_rr= rr.predict(X_train)
    print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
    print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

    pred_test_rr= rr.predict(X_test)
    print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
    print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)

    # Calculate the absolute errors
    errors = abs(pred_test_rr - y_test)

    # Print out the mean absolute error (mae)
    print('Mean Absolute Error:', np.mean(errors), 'degrees.')



    # Calculate mean absolute percentage error (MAPE)
    mape = 100 * (errors / y_test)
    # Calculate and display accuracy
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 2), '%.')
    print("\n")
    print("\n")

Summary for the Eco District ID: 697


C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Train MSE:  92.26475182846802
R squared training set:  80.23738533516618
Test MSE:  117.49643664153372
R squared test set:  76.66075982220899
Mean Absolute Error: YieldKgAcre    95.776049
dtype: float64 degrees.
Accuracy: YieldKgAcre    88.19
dtype: float64 %.




Summary for the Eco District ID: 695


C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Train MSE:  50.36281590526053
R squared training set:  88.23430578956376
Test MSE:  73.11520215990265
R squared test set:  77.19528499791099
Mean Absolute Error: YieldKgAcre    60.237586
dtype: float64 degrees.
Accuracy: YieldKgAcre    93.85
dtype: float64 %.




Summary for the Eco District ID: 698


...

Train MSE:  51.405027914878964
R squared training set:  94.42114493957409
Test MSE:  178.382368470144
R squared test set:  50.080686816433925
Mean Absolute Error: YieldKgAcre    143.614512
dtype: float64 degrees.
Accuracy: YieldKgAcre    71.76
dtype: float64 %.




Summary for the Eco District ID: 848
Train MSE:  64.55769732989704
R squared training set:  90.33175717042786
Test MSE:  103.18076332659875
R squared test set:  66.5860072790304
Mean Absolute Error: YieldKgAcre    82.30503
dtype: float64 degrees.
Accuracy: YieldKgAcre    88.57
dtype: float64 %.




Summary for the Eco District ID: 652
Train MSE:  28.70399514683778
R squared training set:  98.6912030990024
Test MSE:  86.28994650893029
R squared test set:  77.37463149420121
Mean Absolute Error:

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


 YieldKgAcre    67.78657
dtype: float64 degrees.
Accuracy: YieldKgAcre    91.94
dtype: float64 %.




Summary for the Eco District ID: 718
Train MSE:  43.42593857129907
R squared training set:  96.53540093411947
Test MSE:  138.1462489886452
R squared test set:  44.44751162099474
Mean Absolute Error: YieldKgAcre    119.4876
dtype: float64 degrees.
Accuracy: YieldKgAcre    83.78
dtype: float64 %.




Summary for the Eco District ID: 686


C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\Mehul Bha

Train MSE:  0.2889206178196729
R squared training set:  99.9998272247727
Test MSE:  239.3636009025442
R squared test set:  -12.500840018374149
Mean Absolute Error: YieldKgAcre    192.089789
dtype: float64 degrees.
Accuracy: YieldKgAcre    78.09
dtype: float64 %.




Summary for the Eco District ID: 720
Train MSE:  4.932685982710488
R squared training set:  99.93397926426118
Test MSE:  156.6876755306399
R squared test set:  23.0525387947258
Mean Absolute Error: YieldKgAcre    140.198162
dtype: float64 degrees.
Accuracy: YieldKgAcre    52.85
dtype: float64 %.






### # Third for loop for modeling for the eco districts in the 3rd list. 

In [40]:
for i in eco_district_ids_list3:     # for loop for which iterates through each eco district id in the 3rd list.
    print("Summary for the Eco District ID:", i)  # printing the eco district ID
    
    
    # The modeling code is similar to what we have done before in this file. 
    
    df9 = dataframe[dataframe['ECODISTRICT_ID']==i]
    df9.drop(['ECODISTRICT_ID'], axis=1, inplace=True)
    # split data into X and y
    x = pd.DataFrame(df9.drop(labels=['YieldKgAcre'], axis=1))
    y = pd.DataFrame(df9['YieldKgAcre'])
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=40)   

    # Run standardization on X variables
    X_train = scale(X_train)
    X_test = scale(X_test)
     
    rr = Ridge(alpha=0.01)
    rr = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=5)
    rr.fit(X_train, y_train) 
    pred_train_rr= rr.predict(X_train)
    print("Train MSE: ", np.sqrt(mean_squared_error(y_train,pred_train_rr)))
    print("R squared training set: ", r2_score(y_train, pred_train_rr)*100)

    pred_test_rr= rr.predict(X_test)
    print("Test MSE: ", np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
    print("R squared test set: ", r2_score(y_test, pred_test_rr)*100)

    # Calculate the absolute errors
    errors = abs(pred_test_rr - y_test)

    # Print out the mean absolute error (mae)
    print('Mean Absolute Error:', np.mean(errors), 'degrees.')



    # Calculate mean absolute percentage error (MAPE)
    mape = 100 * (errors / y_test)
    # Calculate and display accuracy
    accuracy = 100 - np.mean(mape)
    print('Accuracy:', round(accuracy, 2), '%.')
    print("\n")
    print("\n")

Summary for the Eco District ID: 811
Train MSE:  2.259945942462853
R squared training set:  99.99334876269245
Test MSE:  184.56667845592872
R squared test set:  55.23924040698831
Mean Absolute Error: YieldKgAcre    160.406923
dtype: float64 degrees.
Accuracy: YieldKgAcre    73.5
dtype: float64 %.




Summary for the Eco District ID: 855
Train MSE:  1.5038022564333173
R squared training set:  99.98903726301795
Test MSE:  98.61064706242355
R squared test set:  57.71973418295067
Mean Absolute Error: YieldKgAcre    85.349931
dtype: float64 degrees.
Accuracy: YieldKgAcre    90.29
dtype: float64 %.




Summary for the Eco District ID: 379
Train MSE:  0.15057708273482312
R squared training set:  99.99997270585291
Test MSE:  239.96571136450584
R squared test set:  -80.91422041888387
Mean Absolute Error: YieldKgAcre    219.129738
dtype: float64 degrees.
Accuracy: YieldKgAcre    78.07
dtype: float64 %.




Summary for the Eco District ID: 647


C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop

Train MSE:  0.22331908923287028
R squared training set:  99.99994586474308
Test MSE:  180.8050054459498
R squared test set:  -50.43112130109042
Mean Absolute Error: YieldKgAcre    152.316491
dtype: float64 degrees.
Accuracy: YieldKgAcre    76.13
dtype: float64 %.




Summary for the Eco District ID: 659
Train MSE:  0.18859021891603053
R squared training set:  99.99994016220529
Test MSE:  199.26835782786264
R squared test set:  48.773725892018795
Mean Absolute Error: YieldKgAcre    193.833892
dtype: float64 degrees.
Accuracy: YieldKgAcre    70.15
dtype: float64 %.




Summary for the Eco District ID: 662


C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-d

Train MSE:  0.10604083095882147
R squared training set:  99.99995717495662
Test MSE:  364.5052921929056
R squared test set:  -125.46391123251146
Mean Absolute Error: YieldKgAcre    282.099143
dtype: float64 degrees.
Accuracy: YieldKgAcre   -70.21
dtype: float64 %.




Summary for the Eco District ID: 668
Train MSE:  0.660710058724972
R squared training set:  99.99990584325825
Test MSE:  508.55592354437016
R squared test set:  -10422.541972935682
Mean Absolute Error: YieldKgAcre    471.368038
dtype: float64 degrees.
Accuracy: YieldKgAcre   -10.49
dtype: float64 %.




Summary for the Eco District ID: 833
Train MSE:  0.1305875679943832
R squared training set:  99.99994886835745
Test MSE:  155.54788401724187
R squared test set:  -27.266891431934393
Mean Absolute Error: YieldKgAcre    142.75108
dtype: float64 degrees.
Accuracy: YieldKgAcre    59.73
dtype: float64 %.




Summary for the Eco District ID: 834
Train MSE:  0.12092114991315477
R squared training set:  99.99997425829564
Test MSE:

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\sklearn\metrics\_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
C:\Users\Mehul Bha